In [32]:
from langchain.agents import initialize_agent, AgentType
from langchain.llms import AzureOpenAI

llm = AzureOpenAI(deployment_name="davinci-003", model_name="text-davinci-003")

In [33]:
import json

from langchain.tools import BaseTool
from typing import Any

class Contact:
    def __init__(self, name, email, interests):
        self.name = name
        self.email = email
        self.interests = interests
    def to_dict(self):
        return {
            'name': self.name,
            'email': self.email,
            'interests': self.interests
        }

known_contacts = [
    Contact("John", "john@example.com", ["python", "machine learning"]),
    Contact("Duncan", "duncan@example.com", ["dotnet", "machine learning"]),
    Contact("Herman", "herman@example.com", ["sales", "marketing"]),
]

class ContactSelector(BaseTool):
    name = "Contact Selector"
    description = (
        "use this tool when you need to find a contact person for a specific topic. "
        "this tools does not accept any arguments, it will just return the contacts. You will need to filter them yourself. "
        "This tool will return a JSON list of contacts with their interests, name and email address."
    )

    def _run(self):
        contacts_dict = [contact.to_dict() for contact in known_contacts]
        return json.dumps(contacts_dict, indent=4)
    async def _arun(self) -> Any:
        raise NotImplementedError("This tool does not support async run")

In [34]:
## Creating Email sending Tool

import smtplib
import os
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from langchain.tools import BaseTool
from typing import Any

addr_from = 'droosma+llm@example.com'

# Define SMTP email server details
smtp_server = 'sandbox.smtp.mailtrap.io'
smtp_user = os.getenv('mailtrap_USER')
smtp_pass = os.environ.get('mailtrap_PASSWORD')

class SendEmail(BaseTool):
    name = "Send Email"
    description = (
        "use this tool when you need to send an email."
        "To use the tool you must pass destination address, subject, content."
        "It takes the following parameters"
        "['destination_address ,'subject', 'content']"
    )

    def _run(
            self,
            destination_address: str,
            subject: str,
            content: str
    ):
        msg = MIMEMultipart('alternative')
        msg['To'] = destination_address
        msg['From'] = addr_from
        msg['Subject'] = subject
        text = content
        # Record the MIME types
        part1 = MIMEText(text, 'plain')
        
        # Attach parts into message container.
        msg.attach(part1)

        # Send email
        s = smtplib.SMTP(smtp_server)
        s.starttls() # Use TLS
        s.login(smtp_user, smtp_pass)
        s.sendmail(addr_from, destination_address, msg.as_string())
        s.quit()
    
    async def _arun(self) -> Any:
        raise NotImplementedError("This tool does not support async run")

In [35]:
tools = [SendEmail(), ContactSelector()]
agent = initialize_agent(tools, llm, agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [36]:
agent.run("I want to contact someone that can help me with my interests in dotnet, can you ask that person to contact me at: toeter@example.com?")



> Entering new AgentExecutor chain...

Action:
```
{
  "action": "Contact Selector",
  "action_input": {}
}
```


Observation: [
    {
        "name": "John",
        "email": "john@example.com",
        "interests": [
            "python",
            "machine learning"
        ]
    },
    {
        "name": "Duncan",
        "email": "duncan@example.com",
        "interests": [
            "dotnet",
            "machine learning"
        ]
    },
    {
        "name": "Herman",
        "email": "herman@example.com",
        "interests": [
            "sales",
            "marketing"
        ]
    }
]
Thought: Found a contact that could help
Action:
```
{
  "action": "Send Email",
  "action_input": {"destination_address": "duncan@example.com", "subject": "Help with dotnet", "content": "Hello Duncan, I'm looking for help with my interests in dotnet. Can you contact me at duncan@roosam.me?"}
}
```

Observation: None
Thought: Email sent
Action:
```
{
  "action": "Final Answer",
  "acti

'I have sent an email to Duncan. He should be able to help you with your interests in dotnet.'